In [9]:
import pandas as pd
import seaborn as sns
from datetime import date

In [10]:
def find_sheet_name(sheet_names):
    iip_sheet_names = list(
        filter(
            lambda x: ("gia" not in x.lower() and "csg" not in x.lower())
            and "xk" in x.lower()
            and ("thang" in x.lower() or "tháng" in x.lower()),
            sheet_names,
        )
    )

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    iip_sheet_names = list(
        filter(
            lambda x: ("gia" not in x.lower() and "csg" not in x.lower())
            and ("xuất khẩu" in x.lower() or "xuat khau" in x.lower())
            and ("thang" in x.lower() or "tháng" in x.lower()),
            sheet_names,
        )
    )

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    iip_sheet_names = list(
        filter(
            lambda x: ("gia" not in x.lower() and "csg" not in x.lower())
            and "xk" in x.lower(),
            sheet_names,
        )
    )

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    iip_sheet_names = list(
        filter(
            lambda x: ("gia" not in x.lower() and "csg" not in x.lower())
            and ("xuất khẩu" in x.lower() or "xuat khau" in x.lower()),
            sheet_names,
        )
    )

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    if len(sheet_names) == 43:
        return "25"

    raise RuntimeError(sheet_names, iip_sheet_names)

In [11]:
en_to_vn = {
    "TOTAL": "TỔNG TRỊ GIÁ",
    "Domestic economic sector": "Khu vực kinh tế trong nước",
    "FDI sector": "Khu vực có vốn đầu tư NN",
    "Crude oil": "Dầu thô",
    "Others": "Hàng hoá khác",
    "MAIN ITEMS": "MẶT HÀNG CHỦ YẾU",
    "Aquatic products": "Thủy sản",
    "Vegetables and fruits": "Rau quả",
    "Cashew nut": "Hạt điều",
    "Coffee": "Cà phê",
    "Tea": "Chè",
    "Pepper": "Hạt tiêu",
    "Rice": "Gạo",
    "Cassava & products": "Sắn và sản phẩm của sắn",
    "Clinker and cement": "Clanhke và xi măng",
    "Crude oil": "Dầu thô",
    "Petrol": "Xăng dầu",
    "Chemicals": "Hóa chất",
    "Chemical products": "Sản phẩm hóa chất",
    "Plastic materials": "Chất dẻo nguyên liệu",
    "Plastic products": "Sản phẩm từ chất dẻo",
    "Rubber": "Cao su",
    "Hand bags, wallets, suitcases, umbrellas": "Túi xách, ví, va li, mũ, ô dù",
    "Wood and products": "Gỗ và sản phẩm gỗ",
    "Paper and paper products": "Giấy và các sản phẩm từ giấy",
    "Textile fibres": "Xơ, sợi dệt các loại",
    "Textiles and garments": "Dệt, may",
    "Shoes and sandals": "Giày dép",
    "Auxiliary materials for textile, clothing, leather and footwear": "Nguyên phụ liệu dệt, may, da, giày",
    "Precious stones, metals and products": "Đá quý, KL quý và sản phẩm",
    "Iron, steel": "Sắt thép",
    "Iron and steel products": "Sản phẩm từ sắt thép",
    "Other basic metals and products": "Kim loại thường khác và sản phẩm",
    "Electronic goods, computers and their parts": "Điện tử, máy tính và linh kiện",
    "Phones all of kinds and their parts": "Điện thoại và linh kiện",
    "Cameras, camcorders and their components": "Máy ảnh, máy quay phim và LK",
    "Machinery, instrument, accessory": "Máy móc thiết bị, dụng cụ PT khác",
    "Electrical wire and cable": "Dây điện và cáp điện",
    "Means of transport and components": "Phương tiện vận tải và phụ tùng",
    "Furniture made of non-wood materials": "Sản phẩm nội thất từ chất liệu khác gỗ",
    "Toys, sports equipment and their parts": "Đồ chơi, dụng cụ thể thao và bộ phận",
}

In [20]:
def get_xk(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    df = df.dropna(axis=1, how="all")

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "Khu vực kinh tế trong nước")
                | (df[df.columns[1]] == "Gạo")
                | (df[df.columns[1]] == "Rice")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    df[df.columns[0]] = (
        df[df.columns[0]]
        .replace(r"\s+", " ", regex=True)
        .replace(r"\n", " ", regex=True)
    )

    df[df.columns[0]] = df[df.columns[0]].str.strip()

    df = df.reset_index(drop=True)

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "TỔNG TRỊ GIÁ")
                    | ((df[df.columns[0]] == "TOTAL"))
                    | ((df[df.columns[0]] == "TỔNG TRỊ GIÁ (*)"))
                    | ((df[df.columns[0]] == "TỔNG TRỊ GIÁ(*)"))
                ].index[0]
            )
        )
    )

    df = df.reset_index(drop=True)

    if len(df[df.columns[0]][df[df.columns[0]] == "TOTAL"].index) > 0:
        df[df.columns[0]] = df[df.columns[0]].map(en_to_vn)

    if len(df.columns) > 12:
        df = df.drop(
            columns=[
                df.columns[idx_column] for idx_column in range(12, len(df.columns))
            ]
        )

    total_row = df[df.columns[0]][
        (df[df.columns[0]] == "TỔNG TRỊ GIÁ")
        | ((df[df.columns[0]] == "TOTAL"))
        | ((df[df.columns[0]] == "TỔNG TRỊ GIÁ (*)"))
        | ((df[df.columns[0]] == "TỔNG TRỊ GIÁ(*)"))
    ].index[0]

    columns_is_nan = [
        df.columns[idx_column]
        for idx_column, is_nan in enumerate(df.loc[total_row].isnull())
        if is_nan == True
    ]

    df = df.drop(columns=columns_is_nan)

    if len(df.columns) > 5:
        df = df.drop(
            columns=[df.columns[idx_column] for idx_column in range(5, len(df.columns))]
        )

    if int(date.split("-")[0]) > 2018 or (
        int(date.split("-")[0]) == 2018 and int(date.split("-")[1]) > 8
    ):
        if len(df.columns) == 5 and int(date.split("-")[1]) == 1:
            df = df.drop(columns=[df.columns[1], df.columns[4]])
            df.columns = ["Ngành", "Total-M", "YOY"]
            df["Total-YOY"] = df["Total-M"]
            df["YTD-YOY"] = df["YOY"]
        elif len(df.columns) == 5:
            df.columns = ["Ngành", "Total-M", "Total-YOY", "YOY", "YTD-YOY"]
        elif len(df.columns) == 4 and int(date.split("-")[1]) == 1:
            df = df.drop(columns=[df.columns[1]])
            df.columns = ["Ngành", "Total-M", "YOY"]
            df["Total-YOY"] = df["Total-M"]
            df["YTD-YOY"] = df["YOY"]
        assert len(df.columns) == 5, df.columns

        df = df[df["Total-M"].notna()].replace("-", None)
        df = df[df["Total-YOY"].notna()].replace("-", None)
        df = df[df["YOY"].notna()].replace("-", None)
        df = df[df["YTD-YOY"].notna()].replace("-", None)

        df["YOY"] = df["YOY"] - 100
        df["YTD-YOY"] = df["YTD-YOY"] - 100
    else:
        df = df.drop(columns=[df.columns[1]])

        if int(date.split("-")[1]) == 1 and len(df.columns) == 4:
            df = df.drop(columns=[df.columns[3]])
            df.columns = ["Ngành", "Total-M", "YOY"]
            df["Total-YOY"] = df["Total-M"]
            df["YTD-YOY"] = df["YOY"]
        elif int(date.split("-")[1]) == 1 and len(df.columns) == 3:
            df.columns = ["Ngành", "Total-M", "YOY"]
            df["Total-YOY"] = df["Total-M"]
            df["YTD-YOY"] = df["YOY"]
        elif len(df.columns) == 4:
            df.columns = ["Ngành", "Total-M", "Total-YOY", "YTD-YOY"]
        else:
            raise RuntimeError

        df = df[df["Total-M"].notna()].replace("-", None)
        df = df[df["Total-YOY"].notna()].replace("-", None)
        df = df[df["YTD-YOY"].notna()].replace("-", None)

        if "YOY" in df.columns:
            df["YOY"] = df["YOY"].replace("-", None) - 100
        df["YTD-YOY"] = df["YTD-YOY"].replace("-", None) - 100

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)

    return df


xl = pd.ExcelFile("datas/2018-08-29-02-Bieu-8.2018-1.xlsx")

get_xk(xl, find_sheet_name(xl.sheet_names), "2018-08-29")

,Ngành,Total-M,Total-YOY,YTD-YOY,Date
0,TỔNG TRỊ GIÁ,20900,155411,14.505585,2018-08-29
1,Khu vực kinh tế trong nước,6050,45109,17.417957,2018-08-29
2,Khu vực có vốn đầu tư NN,14850,110302,13.355771,2018-08-29
3,Dầu thô,250,1507,-24.6,2018-08-29
4,Hàng hoá khác,14600,108795,14.151424,2018-08-29
6,Thủy sản,800,5525.905009,6.444198,2018-08-29
7,Rau quả,360,2689.862626,14.753942,2018-08-29
8,Hạt điều,296.224178,2279.070837,2.537997,2018-08-29
9,Cà phê,246.209019,2497.892295,-3.078858,2018-08-29
10,Chè,19.593154,130.744511,-8.596667,2018-08-29


In [21]:
import os
import contextlib

dfs_xk_origin = []
for file_name in sorted(
    os.listdir("../datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    if file_name.startswith(".~"):
        continue

    try:
        file_path = os.path.join("../datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2014:
            continue

        xl = pd.ExcelFile(file_path)

        df_xk = get_xk(xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}")

        dfs_xk_origin.append(df_xk)
    except Exception as e:
        print(file_name)
        raise e

In [26]:
dfs_xk = pd.concat(dfs_xk_origin, ignore_index=True, axis=0)
dfs_xk["Date"] = pd.to_datetime(dfs_xk["Date"], dayfirst=False)
dfs_xk

,Ngành,Total-M,Total-YOY,YOY,YTD-YOY,Date
0,TỔNG TRỊ GIÁ,34010,93060,14.191012,16.983166,2024-03-29
1,Khu vực kinh tế trong nước,9539.525713,25209.707131,21.246905,26.197656,2024-03-29
2,Khu vực có vốn đầu tư NN,24470.474287,67850.292869,11.657896,13.893337,2024-03-29
3,Dầu thô,270.474287,606.729994,55.901562,24.518869,2024-03-29
4,Hàng hoá khác,24200,67243.562875,11.304855,13.805713,2024-03-29
...,...,...,...,...,...,...
4351,"Điện tử, máy tính và LK",700,700,-20.404707,-20.404707,2014-01-29
4352,Điện thoại các loại và linh kiện,1350,1350,-8.878895,-8.878895,2014-01-29
4353,"Máy móc, thiết bị, dụng cụ PT khác",480,480,-6.245577,-6.245577,2014-01-29
4354,Dây điện và cáp điện,50,50,5.076769,5.076769,2014-01-29


In [28]:
len(dfs_xk["Ngành"].unique())

58

In [30]:
convert_nganh = {
    "TỔNG TRỊ GIÁ": "TỔNG TRỊ GIÁ",
    "Khu vực kinh tế trong nước": "Khu vực kinh tế trong nước",
    "Khu vực có vốn đầu tư NN": "Khu vực có vốn đầu tư NN",
    "Dầu thô": "Dầu thô",
    "Hàng hoá khác": "Hàng hoá khác",
    "Thủy sản": "Thủy sản",
    "Rau quả": "Rau quả",
    "Hạt điều": "Hạt điều",
    "Cà phê": "Cà phê",
    "Chè": "Chè",
    "Hạt tiêu": "Hạt tiêu",
    "Gạo": "Gạo",
    "Sắn và sản phẩm của sắn": "Sắn và sản phẩm của sắn",
    "Clanhke và xi măng": "Clanhke và xi măng",
    "Xăng dầu": "Xăng dầu",
    "Hóa chất": "Hóa chất",
    "Sản phẩm hóa chất": "Sản phẩm hóa chất",
    "Chất dẻo nguyên liệu": "Chất dẻo nguyên liệu",
    "Sản phẩm từ chất dẻo": "Sản phẩm từ chất dẻo",
    "Cao su": "Cao su",
    "Túi xách, ví, va li, mũ, ô dù": "Túi xách, ví, va li, mũ, ô dù",
    "Gỗ và sản phẩm gỗ": "Gỗ và sản phẩm gỗ",
    "Giấy và các sản phẩm từ giấy": "Giấy và các sản phẩm từ giấy",
    "Xơ, sợi dệt các loại": "Xơ, sợi dệt các loại",
    "Dệt, may": "Dệt, may",
    "Giày dép": "Giày dép",
    "Nguyên phụ liệu dệt, may, da, giày": "Nguyên phụ liệu dệt, may, da, giày",
    "Sắt thép": "Sắt thép",
    "Sản phẩm từ sắt thép": "Sản phẩm từ sắt thép",
    "Kim loại thường khác và sản phẩm": "Kim loại thường khác và sản phẩm",
    "Điện tử, máy tính và linh kiện": "Điện tử, máy tính và linh kiện",
    "Điện thoại và linh kiện": "Điện thoại và linh kiện",
    "Máy ảnh, máy quay phim và LK": "Máy ảnh, máy quay phim và LK",
    "Máy móc thiết bị, dụng cụ PT khác": "Máy móc thiết bị, dụng cụ PT khác",
    "Dây điện và cáp điện": "Dây điện và cáp điện",
    "Phương tiện vận tải và phụ tùng": "Phương tiện vận tải và phụ tùng",
    "SP nội thất từ chất liệu khác gỗ": "SP nội thất từ chất liệu khác gỗ",
    "Đồ chơi, dụng cụ thể thao và bộ phận": "Đồ chơi, dụng cụ thể thao và bộ phận",
    "Máy ảnh, máy quay phim và linh kiện": "Máy ảnh, máy quay phim và LK",
    "Máy móc thiết bị, dụng cụ phụ tùng khác": "Máy móc thiết bị, dụng cụ PT khác",
    "Kim loại thường khác và SP": "Kim loại thường khác và sản phẩm",
    "Điện tử, máy tính và LK": "Điện tử, máy tính và linh kiện",
    "Điện thoại và LK": "Điện thoại và linh kiện",
    "Máy móc thiết bị, DC PT khác": "Máy móc thiết bị, dụng cụ PT khác",
    "Than đá": "Than đá",
    "Nguyên phụ liệu dệt may, da giầy": "Nguyên phụ liệu dệt, may, da, giày",
    "Đá quý, KL quý và sản phẩm": "Đá quý, KL quý và sản phẩm",
    "Sản phẩm nội thất từ chất liệu khác gỗ": "SP nội thất từ chất liệu khác gỗ",
    "TỔNG TRỊ GIÁ (*)": "TỔNG TRỊ GIÁ",
    "Sản phẩm mây tre, cói, thảm": "Sản phẩm mây tre, cói, thảm",
    "Sản phẩm gốm sứ": "Sản phẩm gốm sứ",
    "Máy móc thiết bị, dụng cụ PT": "Máy móc thiết bị, dụng cụ PT khác",
    "Điện thoại các loại và linh kiện": "Điện thoại và linh kiện",
    "Máy móc, thiết bị, dụng cụ PT khác": "Máy móc thiết bị, dụng cụ PT khác",
    "Sản phẩm chất dẻo": "Sản phẩm từ chất dẻo",
    "Máy móc, thiết bị, DC, PT khác": "Máy móc thiết bị, dụng cụ PT khác",
    "Hàng dệt, may": "Dệt, may",
    "Máy móc, thiết bị, dụng cụ PT": "Máy móc thiết bị, dụng cụ PT khác",
    "TỔNG TRỊ GIÁ(*)": "TỔNG TRỊ GIÁ",
    "Điện tử, máy tính": "Điện tử, máy tính và linh kiện",
}

dfs_xk["Ngành"] = dfs_xk["Ngành"].map(convert_nganh)
dfs_xk

,Ngành,Total-M,Total-YOY,YOY,YTD-YOY,Date
0,TỔNG TRỊ GIÁ,34010,93060,14.191012,16.983166,2024-03-29
1,Khu vực kinh tế trong nước,9539.525713,25209.707131,21.246905,26.197656,2024-03-29
2,Khu vực có vốn đầu tư NN,24470.474287,67850.292869,11.657896,13.893337,2024-03-29
3,Dầu thô,270.474287,606.729994,55.901562,24.518869,2024-03-29
4,Hàng hoá khác,24200,67243.562875,11.304855,13.805713,2024-03-29
...,...,...,...,...,...,...
4351,"Điện tử, máy tính và linh kiện",700,700,-20.404707,-20.404707,2014-01-29
4352,Điện thoại và linh kiện,1350,1350,-8.878895,-8.878895,2014-01-29
4353,"Máy móc thiết bị, dụng cụ PT khác",480,480,-6.245577,-6.245577,2014-01-29
4354,Dây điện và cáp điện,50,50,5.076769,5.076769,2014-01-29


In [31]:
dfs_xk.to_csv('csv/xk.csv')

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_xk[
    (
        (dfs_xk["Ngành"] == "TỔNG TRỊ GIÁ")
        | (dfs_xk["Ngành"] == "Khu vực kinh tế trong nước")
        | (dfs_xk["Ngành"] == "Khu vực có vốn đầu tư NN")
    )
    & (dfs_xk["Date"] > "2015-01-01")
]

fig = px.area(df_show, x="Date", y="YOY", color="Ngành")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_xk[
    (
        (dfs_xk["Ngành"] == "Điện tử, máy tính và linh kiện")
        # | (dfs_xk["Ngành"] == "Điện thoại và linh kiện")
        # | (dfs_xk["Ngành"] == "Máy móc thiết bị, dụng cụ PT khác")
        # | (dfs_xk["Ngành"] == "Dệt, may")
        # | (dfs_xk["Ngành"] == "Giày dép")
        # | (dfs_xk["Ngành"] == "Thủy sản")
        # | (dfs_xk["Ngành"] == "Sắt thép")
        # | (dfs_xk["Ngành"] == "Rau quả")
        # | (dfs_xk["Ngành"] == "Hạt điều")
        # | (dfs_xk["Ngành"] == "Cà phê")
        # | (dfs_xk["Ngành"] == "Chè")
        # | (dfs_xk["Ngành"] == "Gạo")
        | (dfs_xk["Ngành"] == "Hóa chất")
        | (dfs_xk["Ngành"] == "Sản phẩm hóa chất")
    )
    & (dfs_xk["Date"] > "2020-01-01")
]

fig = px.area(df_show, x="Date", y="YOY", color="Ngành")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()